# Item Based Collaborative Movie Recommended system

Here we will use cosine similarity matrix through KNN(k nearest neighbor) technique to recommend the movies based on the previous movie watched.

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb


In [3]:

movies=pd.read_csv('movies.csv',usecols=['movieId','title'])
ratings=pd.read_csv('ratings.csv',usecols=['userId','movieId','rating'])

In [4]:
movies.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [5]:
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [6]:

data=pd.merge(ratings,movies,on='movieId')

In [7]:
data.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)


In [8]:
data.dropna(inplace=True)

In [9]:
a=pd.DataFrame(data.groupby('title')['rating'].count())
a.rename(columns={'rating':'TotalRatingCount'},inplace=True)
a=a.reset_index()

In [10]:
a.head()

,title,TotalRatingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [11]:
df=pd.merge(data,a,on='title')

In [12]:
df.head()

,userId,movieId,rating,title,TotalRatingCount
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215


In [13]:

df=df[df['TotalRatingCount']>50]

In [14]:
df.shape

(40712, 5)

In [15]:
data=pd.pivot_table(data=data,index='title',columns='userId',values='rating').fillna(0)

In [16]:
data.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
from scipy.sparse import csr_matrix
data_matrix=csr_matrix(data.values)

In [18]:
from sklearn.neighbors import NearestNeighbors
model_knn=NearestNeighbors(metric='cosine',algorithm='brute')
model_knn.fit(data_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [19]:
random_choice=np.random.choice(data.shape[0])
random_choice

5150

In [20]:
distances,indices=model_knn.kneighbors(data.iloc[random_choice,:].values.reshape(1,-1),n_neighbors=11)

In [21]:

data.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Top 10 movies recommended moveis->

In [23]:
for i in range(0,len(distances.flatten())):
    if (i==0):
        print('recommendation for {0}:\n'.format(data.index[random_choice]))
    else:
        print('{0}:{1} with distance of {2}:'.format(i,data.index[indices.flatten()[i]],distances.flatten()[i]))

recommendation for Lives of Others, The (Das leben der Anderen) (2006):

1:City of God (Cidade de Deus) (2002) with distance of 0.5961186853609409:
2:Into the Wild (2007) with distance of 0.6221143622032987:
3:Capote (2005) with distance of 0.6230918811202053:
4:Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964) with distance of 0.6237308425851835:
5:Downfall (Untergang, Der) (2004) with distance of 0.6309586843093449:
6:Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001) with distance of 0.633998092653763:
7:Letters from Iwo Jima (2006) with distance of 0.6358379550734665:
8:Annie Hall (1977) with distance of 0.6404121977258188:
9:Persona (1966) with distance of 0.6445903213218522:
10:Strada, La (1954) with distance of 0.6468869775757004:
